# NumPy for Matlab users

### Introduction
MATLAB® and NumPy/SciPy have a lot in common. But there are many differences. NumPy and SciPy were created to do numerical and scientific computing in the most natural way with Python, not to be MATLAB® clones. This page is intended to be a place to collect wisdom about the differences, mostly for the purpose of helping proficient MATLAB® users become proficient NumPy and SciPy users.

### Some Key Differences
| MATLAB| NumPy|
|---|---|
|In MATLAB®, the basic data type is a multidimensional array of double precision floating point numbers. Most expressions take such arrays and return such arrays. Operations on the 2-D instances of these arrays are designed to act more or less like matrix operations in linear algebra.|n NumPy the basic type is a multidimensional array. Operations on these arrays in all dimensionalities including 2D are elementwise operations. However, there is a special matrix type for doing linear algebra, which is just a subclass of the array class. Operations on matrix-class arrays are linear algebra operations.|
|MATLAB® uses 1 (one) based indexing. The initial element of a sequence is found using a(1). See note INDEXING|Python uses 0 (zero) based indexing. The initial element of a sequence is found using a[0].|
|MATLAB®’s scripting language was created for doing linear algebra. The syntax for basic matrix operations is nice and clean, but the API for adding GUIs and making full-fledged applications is more or less an afterthought.| NumPy is based on Python, which was designed from the outset to be an excellent general-purpose programming language. While Matlab’s syntax for some array manipulations is more compact than NumPy’s, NumPy (by virtue of being an add-on to Python) can do many things that Matlab just cannot, for instance subclassing the main array type to do both array and matrix math cleanly.|
|In MATLAB®, arrays have pass-by-value semantics, with a lazy copy-on-write scheme to prevent actually creating copies until they are actually needed. Slice operations copy parts of the array.| In NumPy arrays have pass-by-reference semantics. Slice operations are views into an array.|


### ‘array’ or ‘matrix’? Which should I use?

#### Short answer

#### Use arrays.

They are the standard vector/matrix/tensor type of numpy. Many numpy functions return arrays, not matrices.
There is a clear distinction between element-wise operations and linear algebra operations.
You can have standard vectors or row/column vectors if you like.
The only disadvantage of using the array type is that you will have to use dot instead of * to multiply (reduce) two tensors (scalar product, matrix vector multiplication etc.).



#### Long answer

NumPy contains both an array class and a matrix class. The array class is intended to be a general-purpose n-dimensional array for many kinds of numerical computing, while matrix is intended to facilitate linear algebra computations specifically. In practice there are only a handful of key differences between the two.

* Operator *, dot(), and multiply():
  * For array, ‘``*``’ means element-wise multiplication, and the dot() function is used for matrix multiplication.
  * For matrix, ‘``*``’ means matrix multiplication, and the multiply() function is used for element-wise multiplication.
* Handling of vectors (one-dimensional arrays)
  * For array, the vector shapes 1xN, Nx1, and N are all different things. Operations like A[:,1] return a one-dimensional array of shape N, not a two-dimensional array of shape Nx1. Transpose on a one-dimensional array does nothing.
  * For matrix, one-dimensional arrays are always upconverted to 1xN or Nx1 matrices (row or column vectors). A[:,1] returns a two-dimensional matrix of shape Nx1.
* Handling of higher-dimensional arrays (ndim > 2)
array objects can have number of dimensions > 2;
matrix objects always have exactly two dimensions.
Convenience attributes
  * array has a .T attribute, which returns the transpose of the data.
  * matrix also has .H, .I, and .A attributes, which return the conjugate transpose, inverse, and asarray() of the matrix, respectively.
* Convenience constructor
  * The array constructor takes (nested) Python sequences as initializers. As in, array([[1,2,3],[4,5,6]]).
  * The matrix constructor additionally takes a convenient string initializer. As in matrix("[1 2 3; 4 5 6]").

There are pros and cons to using both:

* array
  * :) You can treat one-dimensional arrays as either row or column vectors. dot(A,v) treats v as a column vector, while dot(v,A) treats v as a row vector. This can save you having to type a lot of transposes.
  * <:( Having to use the dot() function for matrix-multiply is messy – dot(dot(A,B),C) vs. A*B*C.
  * :) Element-wise multiplication is easy: A*B.
  * :) array is the “default” NumPy type, so it gets the most testing, and is the type most likely to be returned by 3rd party code that uses NumPy.
  * :) Is quite at home handling data of any number of dimensions.
  * :) Closer in semantics to tensor algebra, if you are familiar with that.
  * :) All operations (*, /, +, - etc.) are elementwise
* matrix
  * :\\ Behavior is more like that of MATLAB® matrices.
  * <:( Maximum of two-dimensional. To hold three-dimensional data you need array or perhaps a Python list of matrix.
  * <:( Minimum of two-dimensional. You cannot have vectors. They must be cast as single-column or single-row matrices.
  * <:( Since array is the default in NumPy, some functions may return an array even if you give them a matrix as an argument. This shouldn’t happen with NumPy functions (if it does it’s a bug), but 3rd party code based on NumPy may not honor type preservation like NumPy does.
  * :) A*B is matrix multiplication, so more convenient for linear algebra.
  * <:( Element-wise multiplication requires calling a function, multipy(A,B).
  * <:( The use of operator overloading is a bit illogical: * does not work elementwise but / does.

The array is thus much more advisable to use.

### Facilities for Matrix Users
NumPy has some features that facilitate the use of the matrix type, which hopefully make things easier for Matlab converts.

* A matlib module has been added that contains matrix versions of common array constructors like ones(), zeros(), empty(), eye(), rand(), repmat(), etc. Normally these functions return arrays, but the matlib versions return matrix objects.
* mat has been changed to be a synonym for asmatrix, rather than matrix, thus making it a concise way to convert an array to a matrix without copying the data.
* Some top-level functions have been removed. For example numpy.rand() now needs to be accessed as numpy.random.rand(). Or use the rand() from the matlib module. But the “numpythonic” way is to use numpy.random.random(), which takes a tuple for the shape, like other numpy functions.

### Table of Rough MATLAB-NumPy Equivalents
The table below gives rough equivalents for some common MATLAB® expressions. **These are not exact equivalents**, but rather should be taken as hints to get you going in the right direction. For more detail read the built-in documentation on the NumPy functions.

Some care is necessary when writing functions that take arrays or matrices as arguments — if you are expecting an array and are given a matrix, or vice versa, then ‘*’ (multiplication) will give you unexpected results. You can convert back and forth between arrays and matrices using

* asarray: always returns an object of type array
* asmatrix or mat: always return an object of type matrix
* asanyarray: always returns an array object or a subclass derived from it, depending on the input. For instance if you pass in a matrix it returns a matrix.

These functions all accept both arrays and matrices (among other things like Python lists), and thus are useful when writing functions that should accept any array-like object.

In [4]:
import numpy as np
import scipy.linalg as la

###### <span style="color:blue">get the number of dimensions of an array</span>
| MATLAB  |  Numpy |
|---|---|
|ndims(a) |ndim(a) or a.ndim|


In [23]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]])
m = np.matrix([[1,2,3], [4,5,6]])
print('a=', a)
print(19*'-')
print('m=', m)
print(19*'-')
print('ndim(a) = ', np.ndim(a))
print('a.ndim = ', a.ndim)
print(19*'-')
print('np.ndim(m) = ', np.ndim(m))
print('m.ndim = ', m.ndim)

a= [[1 2 3]
 [4 5 6]
 [7 8 9]]
-------------------
m= [[1 2 3]
 [4 5 6]]
-------------------
ndim(a) =  2
na.ndim =  2
-------------------
np.ndim(m) =  2
m.ndim =  2


###### <span style="color:blue"> get the number of elements of an array</span>

| MATLAB  |  Numpy |
|---|---|
|numel(a) |size(a) or a.size|


In [24]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]])
m = np.matrix([[1,2,3], [4,5,6]])
print('a=', a)
print(19*'-')
print('m=', m)
print(19*'-')
print('size(a) = ', np.size(a))
print('a.size = ', a.size)
print(19*'-')
print('np.size(m) = ', np.size(m))
print('m.size = ', m.size)

a= [[1 2 3]
 [4 5 6]
 [7 8 9]]
-------------------
m= [[1 2 3]
 [4 5 6]]
-------------------
size(a) =  9
na.size =  9
-------------------
np.size(m) =  6
m.size =  6


###### <span style="color:blue">get the “size” of the matrix</span>
| MATLAB  |  Numpy |
|---|---|
|size(a) |shape(a) or a.shape|

In [25]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]])
m = np.matrix([[1,2,3], [4,5,6]])
print('a=', a)
print(19*'-')
print('m=', m)
print(19*'-')
print('shape(a) = ', np.shape(a))
print('a.shape = ', a.shape)
print(19*'-')
print('np.shape(m) = ', np.shape(m))
print('m.shape = ', m.shape)

a= [[1 2 3]
 [4 5 6]
 [7 8 9]]
-------------------
m= [[1 2 3]
 [4 5 6]]
-------------------
shape(a) =  (3, 3)
na.shape =  (3, 3)
-------------------
np.shape(m) =  (2, 3)
m.shape =  (2, 3)


###### <span style="color:blue">get the number of elements of the n-th dimension of array a. (Note that MATLAB® uses 1 based indexing while Python uses 0 based indexing, See note [INDEXING](https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html#numpy-for-matlab-users-notes)</span>
| MATLAB  |  Numpy |
|---|---|
|size(a,n) | a.shape[n-1]|


In [28]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]])
m = np.matrix([[1,2,3], [4,5,6]])
print('a=', a)
print(19*'-')
print('m=', m)
print(19*'-')
print('a.shape[2-1] = ', a.shape[2-1])
print(19*'-')
print('m.shape[2-1] = ', m.shape[2-1])

a= [[1 2 3]
 [4 5 6]
 [7 8 9]]
-------------------
m= [[1 2 3]
 [4 5 6]]
-------------------
a.shape[2-1] =  3
-------------------
m.shape[2-1] =  3


###### <span style="color:blue">2x3 matrix literal</span>
| MATLAB  |  Numpy |
|---|---|
|[ 1 2 3; 4 5 6 ] | array([[1.,2.,3.], [4.,5.,6.]])|

In [29]:
a = np.array([[1.,2.,3.], [4.,5.,6.]])
m = np.matrix([[1.,2.,3.], [4.,5.,6.]])
print('a=', a)
print(19*'-')
print('m=', m)

a= [[ 1.  2.  3.]
 [ 4.  5.  6.]]
-------------------
m= [[ 1.  2.  3.]
 [ 4.  5.  6.]]


###### <span style="color:blue">construct a matrix from blocks a, b, c, and d</span>
| MATLAB  |  Numpy |
|---|---|
|[ a b; c d ]	 | vstack([hstack([a,b]), hstack([c,d])]) or bmat('a b; c d').A|

In [ ]:
a = np.array([[1.,2.,3.], [4.,5.,6.]])
b = np.array([[10.,20.,30.], [40.,50.,60.]])
c = np.array([[11.,12.,13.], [14.,15.,16.]])
d = np.array([[19.,29.,39.], [49.,59.,69.]])


m = np.matrix([[1.,2.,3.], [4.,5.,6.]])
n = np.matrix([[10.,20.,30.], [40.,50.,60.]])
r = np.matrix([[11.,12.,13.], [14.,15.,16.]])
p = np.matrix([[19.,29.,39.], [49.,59.,69.]])

In [32]:
e = np.vstack([np.hstack([a,b]), np.hstack([c,d])])
q = np.vstack([np.hstack([m,n]), np.hstack([r,p])])

print ('e = ', e)
print ('q = ', q)

e =  [[  1.   2.   3.  10.  20.  30.]
 [  4.   5.   6.  40.  50.  60.]
 [ 11.  12.  13.  19.  29.  39.]
 [ 14.  15.  16.  49.  59.  69.]]
q =  [[  1.   2.   3.  10.  20.  30.]
 [  4.   5.   6.  40.  50.  60.]
 [ 11.  12.  13.  19.  29.  39.]
 [ 14.  15.  16.  49.  59.  69.]]


In [35]:
f = np.bmat('a b; c d').A
s = np.bmat('m n; r p').A

print ('f = ', f)
print ('s = ', s)

f =  [[  1.   2.   3.  10.  20.  30.]
 [  4.   5.   6.  40.  50.  60.]
 [ 11.  12.  13.  19.  29.  39.]
 [ 14.  15.  16.  49.  59.  69.]]
s =  [[  1.   2.   3.  10.  20.  30.]
 [  4.   5.   6.  40.  50.  60.]
 [ 11.  12.  13.  19.  29.  39.]
 [ 14.  15.  16.  49.  59.  69.]]


###### <span style="color:blue">access last element in the 1xn matrix a</span>
| MATLAB  |  Numpy |
|---|---|
|a(end)	 | a[-1]|

In [40]:
a = np.array([1,2,3,4])
print(a[-1])

4


###### <span style="color:blue">access last element in the 1xn matrix a</span>
| MATLAB  |  Numpy |
|---|---|
|a(2,5)	 | a[1,4]|

In [46]:
a = np.array([[1.,2.,3.,4.,5.], [6.,7.,8.,9.,10.],[11.,12.,13.,14.,15.]])
m = np.matrix([[1.,2.,3.,4.,5.], [6.,7.,8.,9.,10.],[11.,12.,13.,14.,15.]])

print('a=', a)
print(39*'-')
print('m=', m)
print(39*'-')
print(a[1,4])
print(39*'-')
print(m[1,4])

a= [[  1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.  10.]
 [ 11.  12.  13.  14.  15.]]
---------------------------------------
m= [[  1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.  10.]
 [ 11.  12.  13.  14.  15.]]
---------------------------------------
10.0
---------------------------------------
10.0


###### <span style="color:blue">entire second row of a</span>
| MATLAB  |  Numpy |
|---|---|
|a(2,:)| a[1] or a[1,:]|

In [50]:
a = np.array([[1.,2.,3.,4.,5.], [6.,7.,8.,9.,10.],[11.,12.,13.,14.,15.]])
m = np.matrix([[1.,2.,3.,4.,5.], [6.,7.,8.,9.,10.],[11.,12.,13.,14.,15.]])

print('a=', a)
print(39*'-')
print('m=', m)
print(39*'-')
print('2nd row of a: a[1] = ', a[1])
print('2nd row of a: a[1:] = ', a[1,:])
print(39*'-')
print('2nd row of m: m[1] = ', m[1])  
print('2nd row of m: m[1:]', m[1,:])

a= [[  1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.  10.]
 [ 11.  12.  13.  14.  15.]]
---------------------------------------
m= [[  1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.  10.]
 [ 11.  12.  13.  14.  15.]]
---------------------------------------
2nd row of a: a[1] =  [  6.   7.   8.   9.  10.]
2nd row of a: a[1:] =  [  6.   7.   8.   9.  10.]
---------------------------------------
2nd row of m: m[1] =  [[  6.   7.   8.   9.  10.]]
2nd row of m: m[1:] [[  6.   7.   8.   9.  10.]]
